#### Below are the required dependencies to be installed for the project

In [1]:
# !pip install plotly
# !pip install dash
# !pip install dash-renderer
# !pip install dash_html_components
# !pip install dach_core_components
#!pip install dash_table

#### Importing all the dependencies needed for the project

In [2]:
import dash
import dash_core_components as dcc
import plotly.graph_objs as go
import dash_html_components as html
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output
import dash_table

#### Loading data into pandas data frame from the gitHub repository:
**"2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"**

In [3]:
#timeseries global data
global_recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
global_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
global_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

#timeseries data of the US
us_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
us_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

#summary data of the cases across different countries
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

#summary data of the cases across different states
us_state_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_state.csv')

#### Data cleaning

In [4]:
# Changing the dataframe column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
us_state_df.columns = map(str.lower, us_state_df.columns)
global_confirmed_df.columns = map(str.lower, global_confirmed_df.columns)
global_death_df.columns = map(str.lower, global_death_df.columns)
global_recovered_df.columns = map(str.lower, global_recovered_df.columns)
us_confirmed_df.columns = map(str.lower, us_confirmed_df.columns)
us_death_df.columns = map(str.lower, us_death_df.columns)

# changing province/state to state and country/region to country
global_confirmed_df = global_confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
global_recovered_df = global_recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
global_death_df = global_death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})

# changing province_state to state and country_region to country
country_df = country_df.rename(columns={'country_region': 'country'})
us_confirmed_df = us_confirmed_df.rename(columns={'province_state': 'state', 'country_region': 'country'})
us_death_df = us_death_df.rename(columns={'province_state': 'state', 'country_region': 'country'})
us_state_df = us_state_df.rename(columns={'country_region': 'country','province_state' : 'state'})

#removing all the states not belonging to the US from the dataframe
us_state_df = us_state_df[us_state_df.country =='US']

#dropping the population column from the dataframe containing timeseries data of the US
us_death_df = us_death_df.drop(columns=['population'])

In [5]:
global_death_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,33,36,36,40,42,43,47,50,57,58
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,26,26,26,27,27,27,27,28,28,30
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,375,384,392,402,407,415,419,425,432,437
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,36,37,37,37,37,40,40,40,40,41
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [6]:
global_recovered_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,131,135,150,166,179,188,188,207,220,228
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,314,327,345,356,385,394,403,410,422,431
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1047,1099,1152,1204,1355,1408,1479,1508,1558,1651
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,235,248,282,309,333,344,344,344,385,398
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6


In [7]:
country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,Australia,2020-04-30 01:32:26,-25.0000,133.0000,6752,91,5708,953,26.520344,NaN,NaN,1.347749,36,AUS
1,Austria,2020-04-30 01:32:26,47.5162,14.5501,15402,580,12779,2043,171.011725,NaN,NaN,3.765745,40,AUT
2,Canada,2020-04-30 01:32:26,60.0010,-95.0010,52831,3152,20253,29426,139.558897,NaN,NaN,5.966194,124,CAN
3,China,2020-04-30 01:32:26,30.5928,114.3055,83944,4637,78472,835,5.976039,NaN,NaN,5.523921,156,CHN
4,Denmark,2020-04-30 01:32:26,56.0000,10.0000,9206,443,6558,2205,158.937800,NaN,NaN,4.812079,208,DNK


In [8]:
us_confirmed_df.head()

,uid,iso2,iso3,code3,fips,admin2,state,country,lat,long_,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,136,136,136,136,139,141,141,141,141,141
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,14,14,14,14,14,14
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1213,1252,1298,1252,1416,1276,1307,1371,1389,1400
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,53,53,53,54,54,54,55,57,57,57


In [9]:
us_death_df.head()

,uid,iso2,iso3,code3,fips,admin2,state,country,lat,long_,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,62,63,64,63,69,77,83,84,84,86
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,3,3,3,3,3,3,3,4,4,4


#### Sorting the states in the US in non-ascending order of confirmed case count

In [10]:
sorted_us_state_df = us_state_df.sort_values('confirmed', ascending= False)
sorted_us_state_df.head()

,state,country,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,testing_rate,hospitalization_rate
36,New York,US,2020-04-30 01:32:30,42.1657,-74.9481,299691,23477,53238.0,276214.0,36.0,1777.418633,872481.0,65397.0,7.833735,84000036,USA,5174.543066,21.821476
34,New Jersey,US,2020-04-30 01:32:30,40.2989,-74.5210,116365,6771,15642.0,109594.0,34.0,1310.093569,241318.0,NaN,5.818760,84000034,USA,2716.875005,NaN
25,Massachusetts,US,2020-04-30 01:32:30,42.2302,-71.5301,60265,3405,NaN,56860.0,25.0,878.015762,265618.0,5758.0,5.650046,84000025,USA,3869.854651,9.554468
17,Illinois,US,2020-04-30 01:32:30,40.3495,-88.9861,50358,2215,NaN,48143.0,17.0,429.102667,256667.0,NaN,4.398507,84000017,USA,2187.070460,NaN
5,California,US,2020-04-30 01:32:30,36.1162,-119.6816,48328,1939,NaN,46389.0,6.0,123.258610,603139.0,NaN,4.012167,84000006,USA,1538.281633,NaN


#### Sorting the countries in the world in non-ascending order of confirmed case count

In [11]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)
sorted_country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-04-30 01:32:26,40.000000,-100.00000,1038451,60876,120444,857131,315.191889,NaN,NaN,5.862193,840,USA
160,Spain,2020-04-30 01:32:26,40.463667,-3.74922,236899,24275,132929,79695,506.683990,NaN,NaN,10.246983,724,ESP
10,Italy,2020-04-30 01:32:26,41.871900,12.56740,203591,27682,71252,104657,336.726505,NaN,NaN,13.596868,380,ITA
6,France,2020-04-30 01:32:26,46.227600,2.21370,166543,24121,49118,93304,255.146375,NaN,NaN,14.483347,250,FRA
16,United Kingdom,2020-04-30 01:32:26,55.000000,-3.00000,166441,26166,857,139418,245.177194,NaN,NaN,15.720886,826,GBR


In [12]:
#selecting the column of interest for displaying the summary table for countries in the world
global_table_columns = sorted_country_df[['country','last_update','confirmed','recovered','deaths','active','incident_rate','mortality_rate']]

#selecting the column of interest for displaying the summary table for states in the US
us_table_columns = sorted_us_state_df[['state','country','last_update','confirmed','recovered','deaths','active','incident_rate','mortality_rate']]

In [13]:
#general colors dictionary to be used for display
colors = {
    'text' : '#AF2406',
    'plot_color' : '#D3D3D3',
    'paper_color' : 'CFE9E8'
}

In [14]:
# total number of confirmed cases, active cases, deaths and recoveries in the world
global_confirmed_total = int(country_df['confirmed'].sum())
global_deaths_total = int(country_df['deaths'].sum())
global_recovered_total = int(country_df['recovered'].sum())
global_active_total = int(country_df['active'].sum())

In [15]:
# total number of confirmed cases, active cases and deaths in the US
us_confirmed_total = int(us_state_df['confirmed'].sum())
us_deaths_total = int(us_state_df['deaths'].sum())
us_active_total = int(us_state_df['active'].sum())

In [16]:
#initializing the app by calling the Dash class of dash
app = dash.Dash()

#### Creating the layout for Dash app - consist of everything that the page displays (content as well as the styling)
* Page title
* page subheading
* Header image
* User input components
* Tables
* Bar graphs
* line graphs
* Scatterplot
* Bubble plots
* Pie Chart
* Geo/Map plots


In [17]:
app.layout = html.Div([
    html.H1(children = "COVID-19 CORONAVIRUS PANDEMIC", # Page Title
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }), # styling of the page title
    html.H3(children = 'Coronavirus spread summary and tracker - World', # Page subheading
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }), # styling of the page subheading
    html.Img(src = "https://www.stjosephshealth.org/images/covid19-header.jpg", # Header image
            style = {
                'width' : '100%',
                'height' : '400px',
                'border': 'thin grey solid'
            }), # styling of the Header image
    html.Br(),
    html.Br(),
    html.Div([
        html.Span(children = 'Confirmed Cases: '+ str(global_confirmed_total), # displaying global total confirmed case count
              style = {
                  'color' : 'blue',
                  'font-size' : '30px'
           }), # styling the display
        html.Span(children = 'Active Cases: '+ str(global_active_total), # displaying global total active case count
             style = {
                  'color' : 'orange',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),  # styling the display
        html.Span(children = 'Deaths: '+ str(global_deaths_total), # displaying global total death count
              style = {
                  'color' : 'red',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }), # styling the display
        html.Span(children = 'Recovered: '+ str(global_recovered_total), # displaying global total recovered case count
              style = {
                  'color' : 'green',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }), # styling the display
        ],
    style = {
          'textAlign' : 'center',
    }), # styling the display
    html.Br(),
    html.Br(),
    html.Div([
         html.Label(children ="number of countries: ", # Slider label
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }), # styling the label
        dcc.Slider( # adding the dash core component for the slider
            id = 'country-slider', # unique id for the slider
            min = 1, # minimum value for the slider
            max = 50, # minimum value for the slider
            value = 10, # current value for the slider
            step = 1, # increasing step value for the slider
            marks = {i : i for i in range(51)} # displaying the step marks
        )]),
    html.Br(),
    html.Br(),
    dash_table.DataTable( # creating the data table using the dash_table component
        id = 'table-world-cases', # unique id for the dash table displaying world data
        columns=[{"name": i, "id": i} for i in global_table_columns],
        style_table={'overflowX': 'scroll',
                    'maxHeight': '425px',
                    'overflowY': 'scroll'}, # styling the overall table
        fixed_rows={ 'headers': True, 'data': 0 },
         style_cell={
        'height': 'auto',
        'minWidth': '120px', 'maxWidth': '140px',
        'whiteSpace': 'normal',
        'textAlign': 'left',
         'backgroundColor': 'rgb(250, 250, 250)',
        }, # styling the table cells containg the data
        style_header={
        'height' : '40px',
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
        }, # styling the table column headers
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        },
        {
            'if': {'column_id': 'country'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'confirmed'},
            'backgroundColor': '#0AD3F7',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'deaths'},
            'backgroundColor': '#AF2406',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'recovered'},
            'backgroundColor': '#3D9970',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'active'},
            'backgroundColor': '#FC7E27',
            'fontWeight': 'bold'     
        }
        ],
    ), # conditional styling for data columns - country, confirmed cases, active cases, recovered cases and death cases
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(# creating the Graph using the dash core component
            id = 'Check-world-cases', # unique id for the bargraph
            figure = {
                'data' : [
                    go.Bar( # 1st bar for cumulative confirmed cases / day
                        name = 'Confirmed Cases',
                        x = np.array(list(global_confirmed_df.iloc[:, 20:].columns)),
                        y = np.sum(np.asarray(global_confirmed_df.iloc[:,4:]),axis = 0),
                    ),
                    go.Bar(  # 2nd bar for cumulative death cases / day
                       name = 'Deaths', 
                       x = np.array(list(global_death_df.iloc[:, 20:].columns)),
                       y = np.sum(np.asarray(global_death_df.iloc[:,4:]),axis = 0),
                    ),
                    go.Bar(  # 3rd bar for cumulative recovered cases / day
                       name = 'Recoveries',
                       x = np.array(list(global_recovered_df.iloc[:, 20:].columns)),
                       y = np.sum(np.asarray(global_recovered_df.iloc[:,4:]),axis = 0),
                    )
                ],
                'layout' : go.Layout(
                    title = "Cases around the world", # graph title
                    xaxis = {'title' : 'Countries'}, # title for the x-axis
                    yaxis = {'title' : 'Cases'}, # title for the y-axis
                    hovermode = 'closest', # hover showing the closes data point value on the mouse tool-tip
                    height=400,
                    barmode = 'stack', # stacking up the 3 bar graphs together / day
                    showlegend = True
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), # styling the displayed bar graph
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Dropdown( # user input dropdown for selecting confirmed/active/death/recovered cases for the pie chart
            id = 'case-type', # unique id for the dropdown
            options = [ # dropdown values
                {'label' : "Confirmed Cases", 'value' : 'confirmed'},
                {'label' : "Active Cases", 'value' : 'active'},
                {'label' : "Death Cases", 'value' : 'deaths'},
                {'label' : "Recovered Cases", 'value' : 'recovered'}
            ],
            value = 'confirmed', # default value in the dropdown
        )],
         style ={
            'width' : '50%'
        }), # styling the dropdown
    html.Br(),
    html.Br(),
    html.Div([ 
        dcc.Graph(# piechart for the world confirmed/active/death/recovered cases
            id = 'piechart-world-cases')], # unique id for the pie chart displaying the world cases
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }),  # styling the displayed pie chart
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-confirmed-cases', # unique id for the scatterplot displaying confirmed cases across the world
            figure = {
                'data' : [
                    go.Scatter(# scatter plot for displaying confirmed cases across the world / country
                        name = 'country',
                        x=list(sorted_country_df.country), # country names on x-axis
                        y=list(sorted_country_df.confirmed), # confirmed case count on y-axis
                        mode='markers',
                        marker = dict(
                            size =  sorted_country_df.confirmed, # relative size of each bubble wrt confirmed cases
                            sizemin = 4, # setting the minimum size of the bubble
                            sizemode='area',
                            sizeref = 2.*max(sorted_country_df.confirmed)/(100**2),
                            color=np.random.randn(300), # assignimg random colors to the bubbles
                            colorscale='Rainbow',), # using the rainbow color scale provided by plotly
                    )],
                'layout' : go.Layout(
                    title = "Confirmed cases around the world", # scatterplot title
                    xaxis = {'title' : 'Countries', # x-axis title
                            'automargin' : True
                            },
                    yaxis = {'title' : 'Confirmed Cases', # y-axis title
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest' # hover showing the closes data point value on the mouse tool-tip
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), # styling the Scatter plot
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-death-cases',  # unique id for the scatterplot displaying deaths around the world
            figure = {
                'data' : [
                    go.Scatter( # scatter plot for displaying death cases across the world / country
                        name = 'country',
                        x=list(sorted_country_df.country), # country names on x-axis
                        y=list(sorted_country_df.deaths), # death case count on y-axis
                        mode='lines', #drawing a line plot
                        marker = dict(
                            color = 'red')
                    )],
                'layout' : go.Layout(
                    title = "Deaths around the world", # scatterplot title
                    xaxis = {'title' : 'Countries', # x-axis title
                            'automargin' : True},
                    yaxis = {'title' : 'Deaths', # y-axis title
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest' # hover showing the closes data point value on the mouse tool-tip
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), # styling the line plot
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-recovery-cases',  # unique id for the scatterplot displaying recoveries across the world
            figure = {
                'data' : [
                    go.Scatter( # scatter plot for displaying recovered cases across the world / country
                        name = 'country',
                        x=list(sorted_country_df.country), # country names on x-axis
                        y=list(sorted_country_df.recovered), # recovered case count on y-axis
                        mode='markers+lines', # drawing a lineplot with markers
                        marker = dict(
                            color = 'green')
                    )],
                'layout' : go.Layout(
                    title = "Recoveries around the world", # scatterplot title
                    xaxis = {'title' : 'Countries', # x-axis title
                            'automargin' : True},
                    yaxis = {'title' : 'Recoveries', # y-axis title
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest' # hover showing the closes data point value on the mouse tool-tip
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), #styling the scatterplot
    html.Br(),
    html.Br(),
    html.Div([
        html.Label(children ="Country (type a country name): ", # label for the text input box
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }), # styling the label
        dcc.Input( # creating a text input box using the dash core component element
            id = 'country-input', # unique id for taking user input for the country name
            type = 'text',
            value = 'World' # default value of the text input
        )],
        style ={
            'width' : '50%'
        }), # styling the input box
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(id = 'Check-country-cases')], # unique id for the line plot displaying confirmed,death and recovered cases
    style = {
        'border' :'solid 1px green',
    }), #styling for the line plot
    html.Br(),
    html.Br(),
    html.Div([
       dcc.Graph(
            id = 'world-geo-case-tracking', # unique id for the geo/map showing world map with the case counts/country
            figure = {
                'data' : [
                    go.Choropleth( # Choropleth component of the graphing object is used to plot the map
                        locations=country_df['country'], #using the country names for datapoints
                        z=country_df['confirmed'].astype('int64'), # defining color scale based on confirmed case count
                        locationmode='country names',
                        colorscale='Reds', #defining the color scale to be used
                        autocolorscale=False,
                         text = country_df['country']  + '<br>' + 'Confirmed: ' + 
                        country_df['confirmed'].astype(str)  + '<br>' +'Active: ' + 
                        country_df['active'].astype(str)  + '<br>' + 'Deaths: ' + 
                        country_df['deaths'].astype(str)  + '<br>' + 'Recovered: ' + 
                        country_df['recovered'].astype(str), # text to be displayed on mouse tooltip when hovered over
                        marker_line_color='black',
                        colorbar_title="COVID19 confirmed cases", # title for the colorbar
                    )],
                'layout' : go.Layout(
                    title_text = "Spread of Coronavirus across the World", # plot title
                    height=500,
                    hovermode = 'closest', # hover showing the closes data point value on the mouse tool-tip
                    geo = dict(
                        scope='world', # plotting the world graph
                        showlakes=True,
                        lakecolor='rgb(255, 255, 255)'
                    ),
                )})],
    style = {
        'border' :'solid 1px green',
    } #styling the plot
    ),
    html.Br(),
     html.H3(children = 'Coronavirus spread summary and tracker - US', # Page subheading
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }), # styling of the page subheading
    html.Div([
        html.Span(children = 'US Confirmed Cases: '+ str(us_confirmed_total), # displaying US total confirmed case count
              style = {
                  'color' : 'blue',
                  'font-size' : '30px'
           }), # styling the display
        html.Span(children = 'US Active Cases: '+ str(us_active_total), # displaying US total active case count
             style = {
                  'color' : 'orange',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }), # styling the display
        html.Span(children = 'US Deaths: '+ str(us_deaths_total), # displaying US total death count
              style = {
                  'color' : 'red',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }), # styling the display
        ],
    style = {
          'textAlign' : 'center',
    }), # styling the display
    html.Br(),
    html.Br(),
    html.Div([
         html.Label(children ="number of states: ", # Slider label
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }), # styling the label
        dcc.Slider( # adding the dash core component for the slider
            id = 'state-slider', # unique id for the slider
            min = 1, # minimum value for the slider
            max = 50, # maximum value for the slider
            value = 10, # current value for the slider
            step = 1, # increasing step value for the slider
            marks = {i : i for i in range(51)}
        )]),
    html.Br(),
    html.Br(),
    dash_table.DataTable( # creating the data table using the dash_table component
        id = 'table-US-cases', # unique id for the dash table displaying world data
        columns=[{"name": i, "id": i} for i in us_table_columns],
        style_table={'overflowX': 'scroll',
                    'maxHeight': '425px',
                    'overflowY': 'scroll'}, # styling the overall table
        fixed_rows={ 'headers': True, 'data': 0 },
         style_cell={
        'height': 'auto',
        'minWidth': '120px', 'maxWidth': '140px',
        'whiteSpace': 'normal',
        'textAlign': 'left',
         'backgroundColor': 'rgb(250, 250, 250)',
        }, # styling the table cells containg the data
        style_header={
        'height' : '40px',
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
        }, # styling the table column headers
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        },
        {
            'if': {'column_id': 'country'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'state'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'confirmed'},
            'color': '#0B53E6',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'deaths'},
            'color': '#AF2406',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'recovered'},
            'color': '#3D9970',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'active'},
            'color': '#FC7E27',
            'fontWeight': 'bold'     
        }
        ],
    ), # conditional styling for data columns - country, confirmed cases, active cases, recovered cases and death cases
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph( # creating the Graph using the dash core component
            id = 'Check-us-cases', # unique id for the bargraph
            figure = {
                'data' : [
                    go.Bar( # 1st bar for cumulative confirmed cases / day
                        name = 'Confirmed Cases',
                        x = np.array(list(us_confirmed_df.iloc[:, 21:].columns)),
                        y = np.sum(np.asarray(us_confirmed_df.iloc[:,12:]),axis = 0),
                    ),
                    go.Bar( # 2nd bar for cumulative death cases / day
                       name = 'Deaths', 
                       x = np.array(list(us_death_df.iloc[:, 25:].columns)),
                       y = np.sum(np.asarray(us_death_df.iloc[:,12:]),axis = 0),
                    ),
                ],
                'layout' : go.Layout(
                    title = "Cases around the US", # graph title
                    xaxis = {'title' : 'States'}, # title for the x-axis
                    yaxis = {'title' : 'Cases'}, # title for the y-axis
                    hovermode = 'closest', # hover showing the closes data point value on the mouse tool-tip
                    height=400,
                    barmode = 'stack', # stacking up the 2 bar graphs together / day
                    showlegend = True
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), # styling the displayed bar graph
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Dropdown( # user input dropdown for selecting confirmed/active/death/recovered cases for the pie chart
            id = 'us-case-type', # unique id for the dropdown
            options = [ # dropdown values
                {'label' : "Confirmed Cases", 'value' : 'confirmed'},
                {'label' : "Active Cases", 'value' : 'active'},
                {'label' : "Death Cases", 'value' : 'deaths'},
            ],
            value = 'confirmed', # default value in the dropdown
        )],
         style ={
            'width' : '50%'
        }), # styling the dropdown
    html.Br(),
    html.Br(),
    html.Div([ 
        dcc.Graph( # piechart for the world confirmed/active/death cases
            id = 'piechart-US-cases')], # unique id for the pie chart displaying the cases in the US
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), # styling the displayed pie chart
        html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-confirmed-cases', # unique id for the scatterplot displaying confirmed cases across the US
            figure = {
                'data' : [
                    go.Scatter(# scatter plot for displaying confirmed cases across the state in the US
                        name = 'state',
                        x=list(sorted_us_state_df.state), # state names on x-axis
                        y=list(sorted_us_state_df.confirmed), # confirmed case count on y-axis
                        mode='markers',
                        marker = dict(
                            size =  sorted_country_df.confirmed, # relative size of each square wrt confirmed cases
                            sizemin = 4, # setting the minimum size of the square
                            sizemode='area',
                            sizeref = 2.*max(sorted_country_df.confirmed)/(100**2),
                            color=np.random.randn(300),  # assignimg random colors to the squares
                            symbol = 'square', # defining the marker to be of type square
                            colorscale='Rainbow',), # using the rainbow color scale provided by plotly
                    )],
                'layout' : go.Layout(
                    title = "Confirmed cases around US", # scatterplot title
                    xaxis = {'title' : 'States', # x-axis title
                            'automargin' : True
                            },
                    yaxis = {'title' : 'Confirmed Cases', # y-axis title
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest' # hover showing the closes data point value on the mouse tool-tip
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), # styling the Scatter plot
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-death-cases', # unique id for the scatterplot displaying deaths around the US
            figure = {
                'data' : [
                    go.Scatter( # scatter plot for displaying death cases across the states in the US
                        name = 'state',
                        x=list(sorted_us_state_df.state),  # state names on x-axis
                        y=list(sorted_us_state_df.deaths), # death case count on y-axis
                        mode='markers+lines', # drawing a line plot
                        marker = dict(
                            color = 'red')
                    )],
                'layout' : go.Layout(
                    title = "Deaths around US", # scatterplot title
                    xaxis = {'title' : 'States', # x-axis title
                            'automargin' : True},
                    yaxis = {'title' : 'Deaths', # y-axis title
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest' # hover showing the closes data point value on the mouse tool-tip
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }), # styling the line plot
    html.Br(),
    html.Br(),
    html.Div([
       dcc.Graph(
            id = 'usa-geo-case-tracking', # unique id for the geo/map showing US map with the case counts/state
            figure = {
                'data' : [
                    go.Scattergeo( # Scattergeo component of the graphing object is used to plot the US map
                        lat = us_state_df['lat'], # using the latitude information from the dataset
                        lon = us_state_df['long_'], # using the longitude information from the dataset
                        text = us_state_df['state']  + '<br>' + 'Confirmed: ' + 
                        us_state_df['confirmed'].astype(str)  + '<br>' +'Active: ' + 
                        us_state_df['active'].astype(str)  + '<br>' + 'Deaths: ' + 
                        us_state_df['deaths'].astype(str)  + '<br>' + 'Recovered: ' + 
                        us_state_df['recovered'].astype(str), # text to be displayed on mouse tooltip when hovered over
                        mode = 'markers',
                        marker = dict(
                            size =  us_state_df.confirmed/5, # relative size of each bubble wrt confirmed cases
                            sizemin = 4,# setting the minimum size of the bubble
                            sizemode='area',
                            sizeref = 2.*max(us_state_df.confirmed)/(100**2),
                            color = 'rgba(135, 206, 250, 0.8)', # defining the fill color for the bubble
                            line = dict(
                            color = 'rgb(250,0,0)', #defining the color for the bubble boundary
                              width = 2 #defining the width of the bubble boundary
                            )
                       ),
                        hoverinfo = 'text',
                        name = 'state'
                    )],
                'layout' : go.Layout(
                    title_text = "Spread of Coronavirus across the US", # plot title
                    height=500,
                    hovermode = 'closest', # hover showing the closes data point value on the mouse tool-tip
                    geo = dict(
                        scope = 'usa', # plotting the US graph by defining the graph scope
                        showcoastlines=True, coastlinecolor="RebeccaPurple",
                        showland=True, landcolor="LightGrey",
                        projection = dict(type = 'albers usa'),
                    )
                )})],
    style = {
        'border' :'solid 1px green',
    } #styling the geo plot
    ),
],
style = {
    'padding' : '20px'
}) #styling the overall layout
    


#### Using the Dash dependencies to process user input for the slider using its id and displaying the world summary table accordingly

In [18]:
@app.callback(
    Output('table-world-cases', 'data'), # updating the data component of the table using the function return value
    [Input('country-slider', 'value')]) # using the slider value as input to generate the output graph
def display_country_cases_table(number):
    #displaying the table with number of rows according to the user input
    return sorted_country_df.head(number).to_dict('records') 

#### Using the Dash dependencies to process user input for the slider using its id and displaying the US summary table accordingly

In [19]:
@app.callback(
    Output('table-US-cases', 'data'), # updating the data component of the table using the function return value
    [Input('state-slider', 'value')]) # using the slider value as input to generate the output graph
def display_country_cases_table(number):
    #displaying the table with number of rows according to the user input
    return sorted_us_state_df.head(number).to_dict('records')

#### Using the Dash dependencies to process user input for the dropdown using its id and then using the value of the dropdown to plot pie chart for the corresponding world cases

In [20]:
@app.callback(
    Output('piechart-world-cases', 'figure'), #updating the figure component of the pie plot using the function return value
    [Input('case-type', 'value')]) # using the dropdown value as input to generate the output pieplot
def plot_world_info(caseType):
    if(caseType == 'confirmed'): # defining the trace for the pie plot for confirmed cases
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.confirmed, # using the confirmed case values from the data set for pie plot
            hole = 0.6,
            hoverinfo = 'label+value+percent', # defining the text to be displayed on mouse tool-tip when hovered
            textposition = 'inside'
        )
    elif(caseType == 'active'): # defining the trace for the pie plot for active cases
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.active, # using the active case values from the data set for pie plot
            hole = 0.6,
            hoverinfo = 'label+value+percent', #defining the text to be displayed on mouse tool-tip when hovered
            textposition = 'inside'
        )
    elif(caseType == 'deaths'): # defining the trace for the pie plot for death cases
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.deaths, # using the death case values from the data set for pie plot
            hole = 0.6,
            hoverinfo = 'label+value+percent', #defining the text to be displayed on mouse tool-tip when hovered
            textposition = 'inside'
        )
    else: # defining the trace for the pie plot for recovered cases
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.recovered, # using the recovered case values from the data set for pie plot
            hole = 0.6,
            hoverinfo = 'label+value+percent', #defining the text to be displayed on mouse tool-tip when hovered
            textposition = 'inside'
        )
    return {
        'data': [trace], # adding the trace to the data component of the pie plot graphing object
        'layout': dict(
            title = str(caseType) + ' cases around the world', # Plot title
            showlegend = True,
            hovermode = 'closest', # hover showing the closes data point value on the mouse tool-tip
            uniformtext_minsize=8, # defining the size of the text displayed on the pie plot
            uniformtext_mode='hide',
        )
    }

#### Using the Dash dependencies to process user input for the dropdown using its id and then using the value of the dropdown to plot pie chart for the corresponding state cases in the US

In [21]:
@app.callback(
    Output('piechart-US-cases', 'figure'), #updating the figure component of the pie plot using the function return value
    [Input('us-case-type', 'value')]) # using the dropdown value as input to generate the output pieplot
def plot_world_info(caseType):
    if(caseType == 'confirmed'): # defining the trace for the pie plot for confirmed cases
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.confirmed, # using the confirmed case values from the data set for pie plot
            hole = 0.6,
            hoverinfo = 'label+value+percent', #defining the text to be displayed on mouse tool-tip when hovered
            textposition = 'inside'
        )
    elif(caseType == 'active'): # defining the trace for the pie plot for active cases
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.active, # using the active case values from the data set for pie plot
            hole = 0.6,
            hoverinfo = 'label+value+percent', #defining the text to be displayed on mouse tool-tip when hovered
            textposition = 'inside'
        )
    elif(caseType == 'deaths'): # defining the trace for the pie plot for death cases
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.deaths, # using the death case values from the data set for pie plot
            hole = 0.6,
            hoverinfo = 'label+value+percent', #defining the text to be displayed on mouse tool-tip when hovered
            textposition = 'inside'
        )
    return {
        'data': [trace], # adding the trace to the data component of the pie plot graphing object
        'layout': dict(
            title = str(caseType) + ' cases around US', # Plot title
            showlegend = True,
            hovermode = 'closest', # hover showing the closes data point value on the mouse tool-tip
            uniformtext_minsize=8, # defining the size of the text displayed on the pie plot
            uniformtext_mode='hide',
        )
    }
    

#### Using the Dash dependencies to process user input for the text input box using its id and then using the value of the inputbox to  to generate country specific cumulative line plot of confirmed cases, deaths and recoveries for the the 

In [22]:
@app.callback(
    Output('Check-country-cases', 'figure'), #updating the figure component of the scatterplot using the function return value
    [Input('country-input', 'value')]) # using the text inputbox value as input to generate the output scatterplot
def plot_country_info(country):
    labels = ['confirmed','deaths','recovered'] # defining labels for each line plot
    colors = ['orange','red','green'] # defining color for each line plot
    mode_size = [7, 7, 7] # defining marker size for each line plot
    line_size = [4, 4, 4] # defining line size for each line plot
    
    df_list = [global_confirmed_df, global_death_df, global_recovered_df] #using the confirmed/death/recovered df to generate the plot
    
    fig = go.Figure(); # defining the figure component of the graph
    
    for i, df in enumerate(df_list):
        if country.lower() == 'world': #default case when the value for the inputbox is world
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0) # displaying the sum of the cases around the world
            
        else: # displaying the cases of the user selected country
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'].str.lower() == country.lower()].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers', #adding the trace to the plot figure
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1]) #text to be displayed on mouse tool-tip when hovered
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country, # Plot title
        xaxis_title='Date', # x-axis title
        yaxis_title='No. of Cases', # y-axis title
        margin=dict(l=20, r=20, t=40, b=20),
        height = 500,        
    );
    return fig

#### Start the dash app server, dash uses localhost as the default app server and we have given the port  number as 4048 when running the server

In [23]:
if __name__ == '__main__':
    app.run_server(port = 4048,debug=False,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4048/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2020 21:58:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 21:58:53] "POST /_dash-update-component